In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Mapa Interativo: Uso do Solo + Valores do Solo - Londrina
Camada 1: Classes de uso do solo (patches 224x224 pixels)
Camada 2: Valores do solo preditos (quadrados 109.45m × 109.45m)
"""

import pandas as pd
import numpy as np
import folium
from folium import plugins
from shapely.geometry import Polygon
from pyproj import Transformer
import matplotlib.colors as mcolors
import os

# ============================================================================
# 1. DEFINIÇÃO DE CORES E PALETAS
# ============================================================================

# Cores para classes de uso do solo
class_colors = {
    'bare': '#bf8040',        # Marrom (solo exposto)
    'bush': '#009900',        # Verde médio
    'crop': '#ffdf99',        # Amarelo claro (agricultura)
    'grass': '#33ff33',       # Verde claro/lima
    'hduf': '#69000d',        # Vermelho escuro (alta densidade urbana)
    'industrial': '#e93529',  # Vermelho alaranjado (industrial)
    'lduf': '#c7171c',        # Vermelho médio (baixa densidade urbana)
    'mduf': '#a10e15',        # Vermelho médio-escuro (média densidade urbana)
    'tree': '#003300',        # Verde escuro
    'water': '#0b9fd5'        # Azul (água)
}

# Paleta tim.colors() do R para valores do solo
def tim_colors(n=1000):
    """Recria a paleta tim.colors() do R"""
    colors = [
        '#000080',  # azul marinho
        '#0000CC',  # azul escuro
        '#0040FF',  # azul
        '#0080FF',  # azul claro
        '#00BFFF',  # azul céu
        '#00FFFF',  # ciano
        '#00FFBF',  # ciano-verde
        '#00FF80',  # verde-ciano
        '#00FF40',  # verde claro
        '#00FF00',  # verde
        '#40FF00',  # verde-amarelo
        '#80FF00',  # amarelo-verde
        '#BFFF00',  # amarelo claro
        '#FFFF00',  # amarelo
        '#FFBF00',  # amarelo-laranja
        '#FF8000',  # laranja
        '#FF4000',  # laranja-vermelho
        '#FF0000'   # vermelho
    ]
    
    cmap = mcolors.LinearSegmentedColormap.from_list("tim", colors, N=n)
    return [mcolors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, n)]

pal_valores = tim_colors(1000)

# ============================================================================
# 2. FUNÇÕES AUXILIARES
# ============================================================================

def parse_coordinates(coord_str):
    """Extrai lat, lon de string 'lat,lon'"""
    lat, lon = map(float, coord_str.split(','))
    return lat, lon

def create_patch_polygon(row):
    """Cria polígono do patch a partir dos 4 vértices"""
    tl_lat, tl_lon = parse_coordinates(row['top_left'])
    tr_lat, tr_lon = parse_coordinates(row['top_right'])
    bl_lat, bl_lon = parse_coordinates(row['bottom_left'])
    br_lat, br_lon = parse_coordinates(row['bottom_right'])
    
    # Folium usa [lat, lon]
    return [
        [tl_lat, tl_lon],  # top_left
        [tr_lat, tr_lon],  # top_right
        [br_lat, br_lon],  # bottom_right
        [bl_lat, bl_lon],  # bottom_left
        [tl_lat, tl_lon]   # fecha o polígono
    ]

def create_square_utm(utm_x, utm_y, half_size):
    """Cria um quadrado em coordenadas UTM"""
    return Polygon([
        (utm_x - half_size, utm_y - half_size),  # inferior esquerdo
        (utm_x + half_size, utm_y - half_size),  # inferior direito
        (utm_x + half_size, utm_y + half_size),  # superior direito
        (utm_x - half_size, utm_y + half_size),  # superior esquerdo
        (utm_x - half_size, utm_y - half_size)   # fecha o polígono
    ])

def transform_polygon_to_latlon(polygon_utm, transformer):
    """Converte polígono UTM para lat/lon"""
    coords_utm = list(polygon_utm.exterior.coords)
    coords_latlon = []
    
    for x_utm, y_utm in coords_utm:
        lon, lat = transformer.transform(x_utm, y_utm)
        coords_latlon.append([lat, lon])  # Folium usa [lat, lon]
    
    return coords_latlon

# ============================================================================
# 3. CARREGAR DADOS
# ============================================================================

print("Carregando dados...")



# Dados de cobertura do solo das 12 cidades
df_uso_lda = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_londrina/inference_results_with_positions_20250609_083252.csv', sep=';')
df_uso_ibip = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_ibipora/inference_results_with_positions_20250609_164529.csv', sep=';')
df_uso_camb = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_cambe/inference_results_with_positions_20250609_161440.csv', sep=';')
df_uso_apuc = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_apucarana/inference_results_with_positions_20251118_114239.csv', sep=';')
df_uso_arap = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_arapongas/inference_results_with_positions_20251118_121155.csv', sep=';')
df_uso_cambir = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_cambira/inference_results_with_positions_20251118_122253.csv', sep=';')
df_uso_jand = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_jandaia/inference_results_with_positions_20251118_122910.csv', sep=';')
df_uso_mand = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_mandaguari/inference_results_with_positions_20251118_125143.csv', sep=';')
df_uso_mari = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_marialva/inference_results_with_positions_20251118_124329.csv', sep=';')
df_uso_rol = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_rolandia/inference_results_with_positions_20251118_130921.csv', sep=';')
df_uso_sar = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_sarandi/inference_results_with_positions_20251118_131430.csv', sep=';')
df_uso_maring = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_maringa/inference_results_with_positions_20251118_133121.csv', sep=';')


# Concatenar dataframes de uso do solo
df_uso = pd.concat([df_uso_lda, df_uso_ibip, df_uso_camb, df_uso_apuc, df_uso_arap, df_uso_cambir, df_uso_jand, df_uso_mand, df_uso_mari, df_uso_rol, df_uso_sar, df_uso_maring], ignore_index=True)
print(f"✓ Total de uso do solo combinado: {len(df_uso):,} patches")

# Dados de valores do solo urbano das 12 cidades
pasta = '/Users/fjcosta/Documents/projects/Tabular/resultados_inferencia/mediana'
arquivos_csv = [
    'predicoes_unit_Sarandi.csv', 
    'predicoes_unit_Rolandia.csv',
    'predicoes_unit_Maringa.csv',
    'predicoes_unit_Marialva.csv',
    'predicoes_unit_Mandaguari.csv',
    'predicoes_unit_Londrina.csv',
    'predicoes_unit_Jandaia.csv',
    'predicoes_unit_Ibipora.csv',
    'predicoes_unit_Cambira.csv',
    'predicoes_unit_Cambe.csv',
    'predicoes_unit_Arapongas.csv',
    'predicoes_unit_Apucarana.csv'
]

dataframes = []
for arquivo in arquivos_csv:
    df = pd.read_csv(os.path.join(pasta, arquivo))
    df['cidade'] = arquivo.replace('predicoes_', '').replace('.csv', '')
    dataframes.append(df)

# Concatenar dataframes de valores do solo
df_valor = pd.concat(dataframes, ignore_index=True)
print(f"✓ Total de valores do solo combinado: {len(df_valor):,} pontos")

# ============================================================================
# 4. PREPARAR CAMADA 1: USO DO SOLO
# ============================================================================

print("\nPreparando camada de uso do solo...")

features_uso = []
for idx, row in df_uso.iterrows():
    if idx % 1000 == 0:
        print(f"  Processando patch {idx+1}/{len(df_uso)}")
    
    polygon_coords = create_patch_polygon(row)
    predicted_class = row['predicted_class']
    color = class_colors.get(predicted_class, '#808080')  # cinza como fallback
    confidence = row['prediction_confidence']
    
    # Converter coordenadas [lat,lon] para [lon,lat] (padrão GeoJSON)
    geojson_coords = [[[coord[1], coord[0]] for coord in polygon_coords]]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": geojson_coords
        },
        "properties": {
            "fillColor": color,
            "color": color,
            "weight": 0,
            "fillOpacity": 0.6,
            "class": predicted_class,
            "confidence": f"{confidence:.2%}",
            "popup": f"Land Use: {predicted_class}<br>Confidence: {confidence:.2%}"
        }
    }
    features_uso.append(feature)

geojson_uso = {
    "type": "FeatureCollection",
    "features": features_uso
}

print(f"✓ Criados {len(features_uso):,} polígonos de uso do solo")

# ============================================================================
# 5. PREPARAR CAMADA 2: VALORES DO SOLO
# ============================================================================

print("\nPreparando camada de valores do solo...")

# Parâmetros do grid
grid_spacing = 109.45  # metros
half_spacing = grid_spacing / 2

# Configurar transformador de coordenadas
transformer = Transformer.from_crs("EPSG:29192", "EPSG:4326", always_xy=True)

# Criar quadrados em UTM
utm_polygons = []
for idx, row in df_valor.iterrows():
    if idx % 1000 == 0:
        print(f"  Criando quadrado {idx+1}/{len(df_valor)}")
    square_utm = create_square_utm(row['utm_x'], row['utm_y'], half_spacing)
    utm_polygons.append(square_utm)

print(f"✓ Criados {len(utm_polygons):,} quadrados em UTM")

# Converter para lat/lon
latlon_polygons = []
for idx, polygon_utm in enumerate(utm_polygons):
    if idx % 1000 == 0:
        print(f"  Convertendo para lat/lon {idx+1}/{len(utm_polygons)}")
    latlon_polygon = transform_polygon_to_latlon(polygon_utm, transformer)
    latlon_polygons.append(latlon_polygon)

print(f"✓ Convertidos {len(latlon_polygons):,} quadrados para lat/lon")

# Obter min e max dos valores preditos (em log)
min_log_price = df_valor['price_predicted'].min()
max_log_price = df_valor['price_predicted'].max()
price_range = max_log_price - min_log_price

# Criar breaks baseados no range dos dados
breaks = np.linspace(min_log_price, max_log_price, 18)

def get_color_from_price(price):
    """Obtém cor da paleta baseada no preço"""
    normalized = (price - min_log_price) / price_range
    color_index = int(normalized * (len(pal_valores) - 1))
    color_index = max(0, min(color_index, len(pal_valores) - 1))
    return pal_valores[color_index]

# Criar features GeoJSON para valores
features_valor = []
for idx in range(len(latlon_polygons)):
    if idx % 1000 == 0:
        print(f"  Criando feature GeoJSON {idx+1}/{len(latlon_polygons)}")
    
    polygon_coords = latlon_polygons[idx]
    price_log = df_valor.iloc[idx]['price_predicted']
    price_real = np.exp(price_log)
    color = get_color_from_price(price_log)
    
    # Converter coordenadas [lat,lon] para [lon,lat] (padrão GeoJSON)
    geojson_coords = [[[coord[1], coord[0]] for coord in polygon_coords]]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": geojson_coords
        },
        "properties": {
            "fillColor": color,
            "color": color,
            "weight": 0,
            "fillOpacity": 0.7,
            "price": f"R$ {price_real:,.2f}/m²",
            "popup": f"Predicted Unit Land Value<br>R$ {price_real:,.2f}/m²"
        }
    }
    features_valor.append(feature)

geojson_valor = {
    "type": "FeatureCollection",
    "features": features_valor
}

print(f"✓ Criados {len(features_valor):,} polígonos de valores")



# ============================================================================
# 6. CRIAR MAPA INTERATIVO
# ============================================================================



########################## MUDAR AQUI #########################
# # Adicionar camada de uso do solo
# layer_uso = folium.FeatureGroup(name='Land Use Classes', show=True)
# folium.GeoJson(
#     geojson_uso,
#     style_function=lambda feature: {
#         'fillColor': feature['properties']['fillColor'],
#         'color': feature['properties']['color'],
#         'weight': feature['properties']['weight'],
#         'fillOpacity': feature['properties']['fillOpacity'],
#         'opacity': 0
#     },
#     tooltip=folium.GeoJsonTooltip(
#         fields=['popup'],
#         aliases=[''],
#         labels=False
#     )
# ).add_to(layer_uso)
# layer_uso.add_to(m)

# # Adicionar camada de valores do solo
# layer_valor = folium.FeatureGroup(name='Land Values', show=True)
# folium.GeoJson(
#     geojson_valor,
#     style_function=lambda feature: {
#         'fillColor': feature['properties']['fillColor'],
#         'color': feature['properties']['color'],
#         'weight': feature['properties']['weight'],
#         'fillOpacity': feature['properties']['fillOpacity'],
#         'opacity': 0
#     },
#     tooltip=folium.GeoJsonTooltip(
#         fields=['popup'],
#         aliases=[''],
#         labels=False
#     )
# ).add_to(layer_valor)
# layer_valor.add_to(m)

# # Adicionar controle de camadas
# folium.LayerControl(position='topleft', collapsed=False).add_to(m)

# print("✓ Camadas adicionadas ao mapa")
#########################################################################


# ============================================================================
# 6. CRIAR MAPA INTERATIVO COM SPLIT VIEW
# ============================================================================

import json

print("\nCriando mapa interativo com split view...")

# Calcular centro do mapa
center_lat = df_uso['center'].apply(lambda x: parse_coordinates(x)[0]).mean()
center_lon = df_uso['center'].apply(lambda x: parse_coordinates(x)[1]).mean()

# Criar mapa base
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)

# Converter GeoJSON para strings JSON
geojson_uso_str = json.dumps(geojson_uso)
geojson_valor_str = json.dumps(geojson_valor)

# CSS para o split view
split_css = """
<style>
.split-slider {
    position: absolute !important;
    top: 0 !important;
    bottom: 0 !important;
    left: 50%;
    width: 40px !important;
    background: transparent !important;
    cursor: ew-resize !important;
    z-index: 9999 !important;
    transform: translateX(-50%);
    pointer-events: all !important;
}

.split-slider-line {
    position: absolute;
    top: 0;
    bottom: 0;
    left: 50%;
    width: 6px;
    background: #fff;
    transform: translateX(-50%);
    box-shadow: 0 0 15px rgba(0,0,0,0.5);
    pointer-events: none;
}

.split-slider-handle {
    position: absolute;
    top: 50%;
    left: 50%;
    transform: translate(-50%, -50%);
    width: 44px;
    height: 44px;
    background: #fff;
    border-radius: 50%;
    box-shadow: 0 2px 10px rgba(0,0,0,0.4);
    display: flex;
    align-items: center;
    justify-content: center;
    font-size: 14px;
    color: #333;
    pointer-events: none;
}

.split-label {
    position: absolute;
    top: 60px;
    padding: 8px 12px;
    background: rgba(255,255,255,0.95);
    border-radius: 4px;
    font-weight: bold;
    font-size: 13px;
    z-index: 9998;
    box-shadow: 0 2px 5px rgba(0,0,0,0.3);
    pointer-events: none;
}

.split-label-left {
    left: 10px;
}

.split-label-right {
    right: 10px;
}

.leaflet-pane.leftPane,
.leaflet-pane.rightPane {
    pointer-events: none;
}

.leaflet-pane.leftPane path,
.leaflet-pane.rightPane path {
    pointer-events: auto;
}
</style>
"""

# JavaScript para o split view
split_js = f"""
<script>
(function() {{
    var dataUso = {geojson_uso_str};
    var dataValor = {geojson_valor_str};
    
    function initSplitView() {{
        var map = null;
        for (var key in window) {{
            try {{
                if (window[key] instanceof L.Map) {{
                    map = window[key];
                    break;
                }}
            }} catch(e) {{}}
        }}
        
        if (!map) {{
            setTimeout(initSplitView, 300);
            return;
        }}
        
        console.log('Mapa encontrado, criando split view...');
        var container = map.getContainer();
        
        // Criar panes
        var leftPane = map.createPane('leftPane');
        var rightPane = map.createPane('rightPane');
        
        leftPane.style.zIndex = 450;
        rightPane.style.zIndex = 451;
        
        // Criar camadas
        var layerUso = L.geoJSON(dataUso, {{
            pane: 'leftPane',
            style: function(feature) {{
                return {{
                    fillColor: feature.properties.fillColor,
                    color: feature.properties.fillColor,
                    weight: 0,
                    fillOpacity: feature.properties.fillOpacity || 0.6
                }};
            }},
            onEachFeature: function(feature, layer) {{
                if (feature.properties.popup) {{
                    layer.bindTooltip(feature.properties.popup);
                }}
            }}
        }}).addTo(map);
        
        var layerValor = L.geoJSON(dataValor, {{
            pane: 'rightPane',
            style: function(feature) {{
                return {{
                    fillColor: feature.properties.fillColor,
                    color: feature.properties.fillColor,
                    weight: 0,
                    fillOpacity: feature.properties.fillOpacity || 0.7
                }};
            }},
            onEachFeature: function(feature, layer) {{
                if (feature.properties.popup) {{
                    layer.bindTooltip(feature.properties.popup);
                }}
            }}
        }}).addTo(map);
        
        console.log('Camadas criadas - Uso:', dataUso.features.length, 'Valor:', dataValor.features.length);
        
        // Criar slider com estrutura melhorada
        var slider = document.createElement('div');
        slider.className = 'split-slider';
        slider.innerHTML = '<div class="split-slider-line"></div><div class="split-slider-handle">◀▶</div>';
        container.appendChild(slider);
        
        // Labels
        var labelLeft = document.createElement('div');
        labelLeft.className = 'split-label split-label-left';
        labelLeft.innerHTML = 'Land Use Classes';
        container.appendChild(labelLeft);
        
        var labelRight = document.createElement('div');
        labelRight.className = 'split-label split-label-right';
        labelRight.innerHTML = 'Land Values';
        container.appendChild(labelRight);
        
        // Estado
        var currentRatio = 0.5;
        var isDragging = false;
        
        function updateClip() {{
            var w = container.offsetWidth;
            var h = container.offsetHeight;
            var clipX = w * currentRatio;
            
            leftPane.style.clip = 'rect(0px, ' + clipX + 'px, ' + h + 'px, 0px)';
            rightPane.style.clip = 'rect(0px, ' + w + 'px, ' + h + 'px, ' + clipX + 'px)';
            slider.style.left = (currentRatio * 100) + '%';
        }}
        
        updateClip();
        
        // Eventos do mapa
        map.on('move zoom resize viewreset', updateClip);
        window.addEventListener('resize', updateClip);
        
        // EVENTOS DO SLIDER - Mouse
        slider.onmousedown = function(e) {{
            isDragging = true;
            e.preventDefault();
            e.stopPropagation();
            console.log('Drag started');
        }};
        
        document.onmousemove = function(e) {{
            if (!isDragging) return;
            var rect = container.getBoundingClientRect();
            currentRatio = Math.max(0.02, Math.min(0.98, (e.clientX - rect.left) / rect.width));
            updateClip();
        }};
        
        document.onmouseup = function() {{
            if (isDragging) {{
                isDragging = false;
                console.log('Drag ended');
            }}
        }};
        
        // EVENTOS DO SLIDER - Touch
        slider.ontouchstart = function(e) {{
            isDragging = true;
            e.preventDefault();
            console.log('Touch drag started');
        }};
        
        document.ontouchmove = function(e) {{
            if (!isDragging) return;
            var touch = e.touches[0];
            var rect = container.getBoundingClientRect();
            currentRatio = Math.max(0.02, Math.min(0.98, (touch.clientX - rect.left) / rect.width));
            updateClip();
        }};
        
        document.ontouchend = function() {{
            if (isDragging) {{
                isDragging = false;
                console.log('Touch drag ended');
            }}
        }};
        
        // Desabilitar drag do mapa quando estiver no slider
        L.DomEvent.disableClickPropagation(slider);
        L.DomEvent.disableScrollPropagation(slider);
        
        console.log('Split view inicializado!');
    }}
    
    if (document.readyState === 'complete') {{
        setTimeout(initSplitView, 500);
    }} else {{
        window.addEventListener('load', function() {{
            setTimeout(initSplitView, 500);
        }});
    }}
}})();
</script>
"""

# Adicionar ao mapa
m.get_root().html.add_child(folium.Element(split_css))
m.get_root().html.add_child(folium.Element(split_js))

print("✓ Split view configurado")
print("  • Lado esquerdo: Land Use Classes")
print("  • Lado direito: Land Values")


# ============================================================================
# 7. CRIAR LEGENDAS
# ============================================================================

print("Criando legendas...")

# Legenda de uso do solo (esquerda)
legend_uso_html = """
<div style="position: fixed; 
            bottom: 30px; 
            left: 10px; 
            width: 200px; 
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(5px);
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.2);
            z-index: 9998; 
            font-family: 'Segoe UI', Roboto, Arial, sans-serif;
            font-size: 12px; 
            padding: 12px 14px;
            border: 1px solid rgba(0,0,0,0.1);">
    <p style="margin: 0 0 10px 0; 
              font-weight: 600; 
              font-size: 13px;
              color: #333;
              border-bottom: 2px solid #4CAF50;
              padding-bottom: 6px;">
        Urban Land Cover Classes (DINOv2 ViTL/14 LoRA)
    </p>
    <div style="display: flex; flex-direction: column; gap: 4px;">
"""

all_classes = {
    'bare': 'Bare Soil',
    'bush': 'Bush',
    'crop': 'Cropland',
    'grass': 'Grassland',
    'hduf': 'High Density Urban',
    'industrial': 'Industrial',
    'lduf': 'Low Density Urban',
    'mduf': 'Medium Density Urban',
    'tree': 'Tree',
    'water': 'Water'
}

for class_key, class_label in all_classes.items():
    color = class_colors[class_key]
    legend_uso_html += f"""
        <div style="display: flex; align-items: center; gap: 8px;">
            <div style="width: 18px; 
                        height: 18px; 
                        background-color: {color}; 
                        border-radius: 3px;
                        border: 1px solid rgba(0,0,0,0.15);
                        flex-shrink: 0;"></div>
            <span style="color: #444; font-size: 11px;">{class_label}</span>
        </div>
    """

legend_uso_html += """
    </div>

      <!-- Rodapé -->
    <div style="margin-top: 10px; 
                padding-top: 8px; 
                border-top: 1px solid #eee;
                text-align: center;">
        <span style="font-size: 9px; color: #888;">
            Northern Paraná, Brazil • 2024
        </span>
    </div>
</div>
"""

# Legenda de valores do solo (direita) com gradiente contínuo
legend_valor_html = f"""
<div style="position: fixed; 
            bottom: 30px; 
            right: 10px; 
            width: 250px; 
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(5px);
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.2);
            z-index: 9998; 
            font-family: 'Segoe UI', Roboto, Arial, sans-serif;
            font-size: 12px; 
            padding: 12px 14px;
            border: 1px solid rgba(0,0,0,0.1);">
    
    <p style="margin: 0 0 10px 0; 
              font-weight: 600; 
              font-size: 13px;
              color: #333;
              border-bottom: 2px solid #4CAF50;
              padding-bottom: 6px;">
        Median Unit Urban Land Value (TabPFN v2)
        <span style="display: block; font-weight: 400; font-size: 10px; color: #666; margin-top: 2px;">
            (R$/m²)
        </span>
    </p>
    
    <!-- Barra de gradiente -->
    <div style="display: flex; align-items: stretch; gap: 8px; margin-bottom: 12px;">
        <div style="width: 20px; 
                    height: 180px; 
                    background: linear-gradient(to bottom, 
                        #FF0000, #FF4000, #FF8000, #FFBF00, #FFFF00, 
                        #BFFF00, #80FF00, #40FF00, #00FF00, #00FF40, 
                        #00FF80, #00FFBF, #00FFFF, #00BFFF, #0080FF, 
                        #0040FF, #0000CC, #000080);
                    border-radius: 4px;
                    border: 1px solid rgba(0,0,0,0.15);"></div>
        <div style="display: flex; 
                    flex-direction: column; 
                    justify-content: space-between;
                    font-size: 10px;
                    color: #555;
                    padding: 2px 0;">
            <span>R$ {np.exp(max_log_price):,.0f}</span>
            <span>R$ {np.exp((max_log_price + min_log_price) * 0.75):,.0f}</span>
            <span>R$ {np.exp((max_log_price + min_log_price) * 0.5):,.0f}</span>
            <span>R$ {np.exp((max_log_price + min_log_price) * 0.25):,.0f}</span>
            <span>R$ {np.exp(min_log_price):,.0f}</span>
        </div>
    </div>
    
   

    <!-- Rodapé -->
    <div style="margin-top: 10px; 
                padding-top: 8px; 
                border-top: 1px solid #eee;
                text-align: center;">
        <span style="font-size: 9px; color: #888;">
            Northern Paraná, Brazil • 2024
        </span>
    </div>
</div>
"""

# Adicionar legendas ao mapa
m.get_root().html.add_child(folium.Element(legend_uso_html))
m.get_root().html.add_child(folium.Element(legend_valor_html))

print("✓ Legendas criadas")







# # ============================================================================
# # 7. CRIAR LEGENDAS
# # ============================================================================

# print("Criando legendas...")

# # Legenda de uso do solo
# legend_uso_html = """
# <div style="position: fixed; 
#            bottom: 50px; left: 10px; width: 200px; 
#            background-color: white; border:2px solid grey; z-index:9999; 
#            font-size:12px; padding: 10px;">
# <p style="margin:0 0 8px 0;"><b>Land Use Classes</b></p>
# <table style="font-size:11px; width:100%;">
# """

# # Adicionar todas as 10 classes com labels adequados
# all_classes = {
#     'bare': 'Bare Soil',
#     'bush': 'Bush',
#     'crop': 'Cropland',
#     'grass': 'Grassland',
#     'hduf': 'High Density Urban',
#     'industrial': 'Industrial',
#     'lduf': 'Low Density Urban',
#     'mduf': 'Medium Density Urban',
#     'tree': 'Tree',
#     'water': 'Water'
# }

# for class_key, class_label in all_classes.items():
#     color = class_colors[class_key]
#     legend_uso_html += f"""
#     <tr>
#         <td style="background-color:{color}; width:20px; height:15px; border:1px solid #666;"></td>
#         <td style="padding-left:8px; vertical-align:middle;">{class_label}</td>
#     </tr>
#     """

# legend_uso_html += """
# </table>
# </div>
# """

# # Legenda de valores do solo
# legend_valor_html = f"""
# <div style="position: fixed; 
#            top: 10px; right: 10px; width: 260px; height: 550px; 
#            background-color: white; border:2px solid grey; z-index:9999; 
#            font-size:12px; padding: 10px; overflow-y: scroll;">
# <p style="margin:0 0 8px 0;"><b>Predicted Unit Land Value<br>(R$/m²)</b></p>
# <table style="font-size:10px; border-collapse: collapse; width:100%;">
# """

# # Adicionar faixas de cores na legenda
# for i in range(len(breaks)-1):
#     color_index = int((i / (len(breaks)-2)) * (len(pal_valores)-1))
#     color = pal_valores[color_index]
    
#     # Calcular preços reais para as faixas
#     price_min = np.exp(breaks[i])
#     price_max = np.exp(breaks[i+1])
    
#     legend_valor_html += f"""
#     <tr>
#         <td style="background-color:{color}; width:20px; height:10px; border:1px solid #ccc;"></td>
#         <td style="padding-left:8px; vertical-align:middle;">R$ {price_min:,.0f} - R$ {price_max:,.0f}</td>
#     </tr>
#     """

# legend_valor_html += f"""
# </table>
# <div style="font-size:10px; margin-top:15px; color:#666; line-height:1.4;">
# <p style="margin:0; font-weight:bold;">Grid Parameters:</p>
# <ul style="margin:5px 0; padding-left:15px;">
#     <li>Land Use: 224×224 pixels patches</li>
#     <li>Land Value: {grid_spacing:.1f}m × {grid_spacing:.1f}m grid</li>
#     <li>Paradigm area: 363m²</li>
# </ul>
# <p style="margin:10px 0 0 0; font-style:italic; font-size:9px;">
# Londrina-PR, Brazil<br>
# Data: 2025
# </p>
# </div>
# </div>
# """

# # Adicionar legendas ao mapa
# m.get_root().html.add_child(folium.Element(legend_uso_html))
# m.get_root().html.add_child(folium.Element(legend_valor_html))

# print("✓ Legendas criadas")

# ============================================================================
# 8. SALVAR MAPA
# ============================================================================

output_file = '/Users/fjcosta/Documents/landCoverlandValue/landcover_landvalue/LandCover_DINO_LandValue_TabPFN.html'
m.save(output_file)

print(f"\n{'='*70}")
print(f"✅ MAPA CRIADO COM SUCESSO!")
print(f"{'='*70}")
# print(f"Arquivo: {output_file}")
print(f"\nCamadas:")
print(f"  • Land Use Classes: {len(features_uso):,} patches")
print(f"  • Land Values: {len(features_valor):,} grid cells")
print(f"\nUse o controle de camadas no mapa para ligar/desligar cada camada.")
print(f"{'='*70}")



Carregando dados...
✓ Total de uso do solo combinado: 73,166 patches
✓ Total de valores do solo combinado: 73,399 pontos

Preparando camada de uso do solo...
  Processando patch 1/73166
  Processando patch 1001/73166
  Processando patch 2001/73166
  Processando patch 3001/73166
  Processando patch 4001/73166
  Processando patch 5001/73166
  Processando patch 6001/73166
  Processando patch 7001/73166
  Processando patch 8001/73166
  Processando patch 9001/73166
  Processando patch 10001/73166
  Processando patch 11001/73166
  Processando patch 12001/73166
  Processando patch 13001/73166
  Processando patch 14001/73166
  Processando patch 15001/73166
  Processando patch 16001/73166
  Processando patch 17001/73166
  Processando patch 18001/73166
  Processando patch 19001/73166
  Processando patch 20001/73166
  Processando patch 21001/73166
  Processando patch 22001/73166
  Processando patch 23001/73166
  Processando patch 24001/73166
  Processando patch 25001/73166
  Processando patch 260